In [67]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import requests
import pandas as pd
from janome.tokenizer import Tokenizer
from tqdm import tqdm

# データ取得


In [ ]:
# なろうapiのエンドポイント
URL = "https://api.syosetu.com/novelapi/api/"

In [48]:
# データ取得

all_data = []
loop_cnt = 0

st_lim_pair = [(1, 499), (500, 500), (1000, 500), (1500, 500), (2000, 500)]
for st, lim in st_lim_pair:
    print("st:", st)  # st：表示開始位置。max2000
    print("lim", lim)  # lim:最大出力数。max500

    params = {
        "of": "s",
        "order": "hyoka",
        "type": "er",  # 完結済み連載小説に絞る
        "out": "json",
        "lim": lim,
        "st": st,
    }  # nコードとあらすじをjsonで出力

    res = requests.get(URL, params=params)
    res = res.json()
    new_data = res[1:]
    all_data.extend(new_data)
    print("done")
    print()
df = pd.DataFrame(all_data)

st: 1
lim 499
done

st: 500
lim 500
done

st: 1000
lim 500
done

st: 1500
lim 500
done

st: 2000
lim 500
done



# doc2vec 用学習データ作成


In [60]:
tokenizer = Tokenizer()

In [86]:
all_story = df.story.to_list()
all_wakati_story = []
for story in tqdm(all_story):
    wakati_story = list(tokenizer.tokenize(story, wakati=True))
    all_wakati_story.append(wakati_story)

100%|██████████| 2499/2499 [00:27<00:00, 89.42it/s] 


# doc2vec 学習・保存


In [94]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(all_wakati_story)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4, epochs=20)

In [102]:
model.save("../model/doc2vec")